In [72]:
%pylab inline

import numpy as np
from pyspark import SparkConf

# import plotly 
# import plotly.plotly as py
# from plotly.graph_objs import Layout
# from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
# plotly.offline.init_notebook_mode()

Populating the interactive namespace from numpy and matplotlib


/Users/kylin1989/anaconda2/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning:

pylab import has clobbered these variables: ['plot']
`%matplotlib` prevents importing * from pylab and numpy



## Creating a DataFrame
Often we are going to be importing data in a CSV format that conforms to some schema. We would like the ability to keep the dimensions and feature headers of that schema. Spark 1.6 has finally included the magic that is Pandas' Dataframes. These are essentially a database table but have built in operations (easily usable with numpy), dataflow optimizations, and easy data exploration.

#### SQLContext
[SQLContext](https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#pyspark.sql.SQLContext) is the main entry point for Spark SQL functionality. A SQLContext can be used create DataFrame, register DataFrame as tables, execute SQL over tables, cache tables, and read parquet files. The SQLContext takes a [SparkContext](https://spark.apache.org/docs/1.6.2/api/python/pyspark.html#pyspark.SparkContext) as a parameter and is in a manner of speaking a child of SparkContext.
```python
train_data_df = sqlContext.read.format('com.databricks.spark.csv').options(header='true',inferSchema='true').load('/FileStore/tables/libwrcg41471805568157/train.csv')
```
This code generates a dataframe by generating a SQLContext. We pass the context some options that inform the Context a header row exists in our CSV and to automatically infer our datatypes. We then load these files from a file I have uploaded to my Databricks Community Edition workspace.

Let's [check the schema](https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html?highlight=printschema#pyspark.sql.DataFrame.printSchema) it has inferred and see if we agree with Spark's assumptions.

In [73]:
train_data_df = (sqlContext
                 .read
                 .format('csv')
                 .options(header='true', inferSchema='true')
                 .load('./data/train.csv'))
train_data_df.cache()
train_data_df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



## What Happens Now?

#### Data Exploration

In [74]:
train_data_df.describe().show()

+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|summary|      PassengerId|           Survived|            Pclass|                Name|   Sex|               Age|             SibSp|              Parch|            Ticket|             Fare|Cabin|Embarked|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|  count|              891|                891|               891|                 891|   891|               714|               891|                891|               891|              891|  204|     889|
|   mean|            446.0| 0.3838383838383838| 2.308641975308642|                null|  null| 29.69911764705882|0.5230078563411896|0.38159371492704824|260318.54916792738| 32.20420

In [75]:
train_data_df
# Same as display(train_data_df) 

DataFrame[PassengerId: int, Survived: int, Pclass: int, Name: string, Sex: string, Age: double, SibSp: int, Parch: int, Ticket: string, Fare: double, Cabin: string, Embarked: string]

#### Data Cleaning
What is something that you noticed that could cause issues?
Hopefully, you noticed that some data objects (objects here are rows which represent a passenger) have a null age. This is going to cause some issues. How can we fix these non-values without affecting any population properties of the data? Let's use some [**domain knowledge**](http://www.simafore.com/blog/the-value-of-domain-knowledge-in-data-science) to do some more data exploration. Let's see if age follows any trends based off of class and/or ticket fare.

In [76]:
explore_age_df = train_data_df.orderBy('Age', ascending=True)

avg_age_df = explore_age_df.where(explore_age_df['Age'].isNotNull()).groupBy('Pclass').avg('Age')
avg_age_df = avg_age_df.select('Pclass', avg_age_df['avg(Age)'].alias('Age'))
avg_age_df

DataFrame[Pclass: int, Age: double]

###### Average Age vs. Class
We've barely done any exploration, and we can already possibly see a correlation between the average age of the passenger and their passenger class. We could easily use these averages to fill in missing data or we can keep looking for greater identifiers to try to eliminate bias.

In [77]:
null_age_df = explore_age_df.where(explore_age_df['Age'].isNull())
null_age_df.describe().show()

+-------+------------------+------------------+------------------+--------------------+------+----+------------------+------------------+------------------+------------------+-----+--------+
|summary|       PassengerId|          Survived|            Pclass|                Name|   Sex| Age|             SibSp|             Parch|            Ticket|              Fare|Cabin|Embarked|
+-------+------------------+------------------+------------------+--------------------+------+----+------------------+------------------+------------------+------------------+-----+--------+
|  count|               177|               177|               177|                 177|   177|   0|               177|               177|               177|               177|   19|     177|
|   mean| 435.5819209039548|0.2937853107344633| 2.598870056497175|                null|  null|null|0.5649717514124294|0.1807909604519774| 202987.1857142857|22.158566666666673| null|    null|
| stddev|250.55290067154587|0.456786974431487

##### Fill missing Age values with the average age per class

In [78]:
from pyspark.sql.functions import when, col

avg_age_list = avg_age_df.collect()

# Replace null values with the average age values from our passenger class list
data_with_age_df = (train_data_df
                     .select('*', 
                             when(train_data_df['Age'].isNull() & (train_data_df['Pclass'] == 1), 
                                  avg_age_list[0].Age)
                             .otherwise(when(train_data_df['Age'].isNull() & (train_data_df['Pclass'] == 1), 
                                             avg_age_list[1].Age)
                                        .otherwise(when(train_data_df['Age'].isNull() & (train_data_df['Pclass'] == 3), 
                                                        avg_age_list[2].Age)
                                                   .otherwise(col('Age')))).alias('FilledAge')))

# Replace the Age column values with those from our FilledAge column and then drop FilledAge.
data_with_age_df = data_with_age_df.withColumn('Age', data_with_age_df['FilledAge']).drop('FilledAge')

###### Index Sex

In [100]:
from pyspark.sql.types import IntegerType
from pyspark.sql import functions
def sex_to_int(sex):
  if(sex.lower() == 'male'):
    return 0
  else:
    return 1
sex_classify = functions.udf(sex_to_int, IntegerType())
sex_int_df = data_with_age_df.select('*', sex_classify(data_with_age_df['Sex']).alias('IntSex'))
data_sex_indexed_df = sex_int_df.withColumn('Sex', sex_int_df['IntSex']).drop('IntSex').cache()

data_sex_indexed_df

DataFrame[PassengerId: int, Survived: int, Pclass: int, Name: string, Sex: int, Age: double, SibSp: int, Parch: int, Ticket: string, Fare: double, Cabin: string, Embarked: string]

###### Index Cabin

In [98]:
def cabin_to_int(cabin):
    if cabin:
        return ord(cabin[0])-ord['A']+1 #A: 1; B:2; C:3; D:4; None:0
    else:
        return 0
cabin_classify = functions.udf(cabin_to_int, IntegerType())

cabin_int_df = data_sex_indexed_df.select('*', cabin_classify(data_sex_indexed_df['Cabin']).alias('IntCabin'))
data_cabin_indexed_df = cabin_int_df.withColumn('Cabin', cabin_int_df['IntCabin']).drop('IntCabin').cache()

data_cabin_indexed_df

DataFrame[PassengerId: int, Survived: int, Pclass: int, Name: string, Sex: int, Age: double, SibSp: int, Parch: int, Ticket: string, Fare: double, Cabin: int, Embarked: string]

In [104]:
def embarked_to_int(embarked):
    if embarked:
        return ord(embarked) #TBD
    else:
        return 0
embarked_classify = functions.udf(embarked_to_int, IntegerType())

embarked_int_df = data_cabin_indexed_df.select('*', embarked_classify(data_sex_indexed_df['Embarked']).alias('IntEmbarked'))
data_embarked_indexed_df = embarked_int_df.withColumn('Embarked', embarked_int_df['IntEmbarked']).drop('IntEmbarked').cache()

data_embarked_indexed_df

DataFrame[PassengerId: int, Survived: int, Pclass: int, Name: string, Sex: int, Age: double, SibSp: int, Parch: int, Ticket: string, Fare: double, Cabin: int, Embarked: int]

In [ ]:
data_embarked_indexed_df

#### Machine Learning

In [101]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler
# training =  clean_data_df.select('Survived','Pclass','Sex','Age','SibSp','Parch','Fare','Cabin','Embarked')
# training

In [122]:
assembler = VectorAssembler(
    inputCols=['Survived','Pclass','Sex','Age','SibSp','Parch','Fare','Cabin','Embarked'], outputCol='Vector')
training = assembler.transform(data_embarked_indexed_df).select(col('Survived').alias('label'),col('Vector').alias('features'))
training

DataFrame[label: int, features: vector]

In [123]:
lr = LogisticRegression()
model1 = lr.fit(training)


Py4JJavaError: An error occurred while calling o3374.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 383.0 failed 1 times, most recent failure: Lost task 0.0 in stage 383.0 (TID 19505, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/kylin1989/Downloads/spark-2.1.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 174, in main
    process()
  File "/Users/kylin1989/Downloads/spark-2.1.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 169, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/Users/kylin1989/Downloads/spark-2.1.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 106, in <lambda>
    func = lambda _, it: map(mapper, it)
  File "/Users/kylin1989/Downloads/spark-2.1.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 92, in <lambda>
    mapper = lambda a: udf(*a)
  File "/Users/